In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define file paths for your CSVs
per_100_poss_file = '../data/raw/NBA24-25PER100STATS.csv'
advanced_stats_file = '../data/raw/NBA24-25ADVANCEDSTATS.csv'
shooting_stats_file = '../data/raw/NBA24-25SHOOTINGSTATS.csv'

# Load each CSV into a Pandas DataFrame
try:
    df_per_100 = pd.read_csv(per_100_poss_file)
    df_advanced = pd.read_csv(advanced_stats_file)
    df_shooting = pd.read_csv(shooting_stats_file)

    print("DataFrames loaded successfully.")
    # ... (rest of your initial print statements for head and columns) ...

except FileNotFoundError as e:
    print(f"Error: One of the CSV files not found. Please ensure they are in the correct directory.")
    print(f"Missing file: {e.filename}")
    exit()

# ... (your existing checks for required_columns and data type conversions) ...


# Now we can merge using the correct column names
if all(col in df_per_100.columns for col in ['Player', 'Team', 'Age']) and \
   all(col in df_advanced.columns for col in ['Player', 'Team', 'Age']):

    merged_df = pd.merge(df_per_100, df_advanced, on=['Player', 'Team', 'Age'], how='inner', suffixes=('_per100', '_adv'))
    print("\n✓ First merge completed successfully")

    if all(col in merged_df.columns for col in ['Player', 'Team', 'Age']) and \
       all(col in df_shooting.columns for col in ['Player', 'Team', 'Age']):

        final_df = pd.merge(merged_df, df_shooting, on=['Player', 'Team', 'Age'], how='inner', suffixes=('', '_shooting'))
        print("✓ Second merge completed successfully")

        print("\n--- Merged DataFrame Info ---")
        print(final_df.info())
        print("\nFinal Merged DataFrame Head (before duplicate handling):")
        print(final_df.head())

        # --- NEW BLOCK START: Handle Duplicate Players (TOT vs. Individual Teams) ---
        print("\n--- Handling Duplicate Players (Multi-Team) ---")

        # Identify players who have 'TOT' in their 'Team' column
        tot_players = final_df[final_df['Team'] == 'TOT']

        # Get the names of players who have a 'TOT' entry
        players_with_tot_entry = tot_players['Player'].unique()

        # Create a DataFrame for players who only played for one team (no 'TOT' entry)
        # This includes players NOT in the players_with_tot_entry list
        # And players who are in the players_with_tot_entry list but are NOT 'TOT' themselves
        single_team_players = final_df[
            (~final_df['Player'].isin(players_with_tot_entry)) |  # Players who never had a TOT entry
            ((final_df['Player'].isin(players_with_tot_entry)) & (final_df['Team'] != 'TOT')) # Players who did, but this isn't their TOT row
        ]

        # Filter single_team_players to only keep actual single-team players
        # For players who had a 'TOT' row, we want to discard their individual team rows.
        # So, we'll only take players from `final_df` who are NOT in `players_with_tot_entry`.
        # This will correctly keep single-team players only.
        final_df_unique_players = final_df[~final_df['Player'].isin(players_with_tot_entry)].copy()

        # Now, add back the 'TOT' rows for the multi-team players
        final_df_unique_players = pd.concat([final_df_unique_players, tot_players]).copy()

        # Sort by Player and ensure no accidental duplicates (e.g., if a TOT row was duplicated during previous merge issues)
        # This will give you one row per player (either their single team or their TOT row).
        final_df_unique_players.drop_duplicates(subset=['Player', 'Age'], keep='first', inplace=True)


        print(f"Original final_df shape: {final_df.shape}")
        print(f"Shape after handling duplicates (should be 1 row per player): {final_df_unique_players.shape}")
        # Overwrite final_df with the cleaned version
        final_df = final_df_unique_players.copy()
        print("Final Merged DataFrame Head (after duplicate handling):")
        print(final_df.head())

        # --- END NEW BLOCK ---


        # --- Initial Data Cleaning and Feature Preparation (rest of your existing blocks) ---

        # 1. Handle redundant/duplicate columns from merging
        # ... (Your existing code for handling redundant columns) ...
        columns_to_drop_after_merge = []
        common_redundant_base_cols = ['Rk', 'G', 'GS', 'MP'] # Add more as you inspect final_df.columns if needed

        for base_col in common_redundant_base_cols:
            if f"{base_col}_adv" in final_df.columns and base_col in final_df.columns:
                columns_to_drop_after_merge.append(f"{base_col}_adv")
            if f"{base_col}_shooting" in final_df.columns and base_col in final_df.columns:
                columns_to_drop_after_merge.append(f"{base_col}_shooting")
            if f"{base_col}_adv" in final_df.columns and f"{base_col}_per100" in final_df.columns:
                columns_to_drop_after_merge.append(f"{base_col}_adv")
            if f"{base_col}_shooting" in final_df.columns and f"{base_col}_per100" in final_df.columns:
                columns_to_drop_after_merge.append(f"{base_col}_shooting")

        if 'Pos_adv' in final_df.columns and 'Pos' in final_df.columns:
            columns_to_drop_after_merge.append('Pos_adv')

        columns_to_drop_after_merge = list(set(columns_to_drop_after_merge))
        final_df.drop(columns=columns_to_drop_after_merge, inplace=True, errors='ignore')
        print(f"\nDataFrame shape after dropping redundant columns: {final_df.shape}")
        print("Columns after dropping redundant ones:")
        print(final_df.columns.tolist())


        # 2. Filter out players with insufficient playing time
        min_mp_threshold = 500

        mp_col_name = 'MP'
        if 'MP_per100' in final_df.columns and 'MP' not in final_df.columns: # Adjust based on what remains
            mp_col_name = 'MP_per100'

        if mp_col_name not in final_df.columns:
            print(f"Error: Could not find a suitable minutes played column ('MP' or 'MP_per100'). Please check your data.")
            exit()

        final_df = final_df[final_df[mp_col_name] >= min_mp_threshold].copy()
        print(f"\nFiltered to players with >= {min_mp_threshold} minutes. New shape: {final_df.shape}")


        # 3. Handling Missing Values (Imputation)
        print("\n--- Missing Value Check (before imputation) ---")
        nan_counts = final_df.isnull().sum()
        print(nan_counts[nan_counts > 0])

        for col in final_df.columns:
            if pd.api.types.is_numeric_dtype(final_df[col]):
                if final_df[col].isnull().any():
                    if any(s in col for s in ['%', 'Ar', 'Dist']) or col in ['PER', 'TS%', 'USG%', 'eFG%', 'FTAr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'BPM', 'VORP']:
                        final_df[col].fillna(0, inplace=True)
                    else:
                        median_val = final_df[col].median()
                        final_df[col].fillna(median_val, inplace=True)
            elif final_df[col].isnull().any():
                final_df[col].fillna('Unknown', inplace=True)

        print("\n--- Missing Value Check (after imputation) ---")
        nan_counts_after = final_df.isnull().sum()
        print(nan_counts_after[nan_counts_after > 0])


        # --- Save the merged and cleaned (unscaled) data ---
        output_cleaned_file = '../data/processed/nba_2025_player_stats_merged_cleaned.csv'
        final_df.to_csv(output_cleaned_file, index=False)
        print(f"\nMerged and cleaned data (unscaled) saved to {output_cleaned_file}")


        # 4. Select features for clustering
        features_for_clustering = [
            'PTS', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'PF',
            'FG%', '3P%', '2P%', 'FT%',
            'eFG%', 'TS%',
            '3PAr', 'FTr',
            'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
            'OWS', 'DWS', 'WS', 'WS/48', 'BPM', 'VORP',
            'Dist.',
            '2P_FG%', '0-3_FG%', '3-10_FG%', '10-16_FG%', '16-3P_FG%',
        ]

        actual_features = [col for col in features_for_clustering if col in final_df.columns]
        missing_features = [col for col in features_for_clustering if col not in final_df.columns]

        if missing_features:
            print(f"\nWarning: The following desired features are missing and will be excluded from clustering: {missing_features}")
            print("Please check your original CSVs and merge process if these are critical.")

        df_clustering = final_df[actual_features].copy()

        # Create player_info DataFrame - Use the correct capitalized names
        player_info_cols = ['Player', 'Team', 'Pos', 'Age'] # Corrected to 'Player', 'Team', 'Pos', 'Age'
        existing_player_info_cols = [col for col in player_info_cols if col in final_df.columns]
        if len(existing_player_info_cols) < len(player_info_cols):
            print(f"Warning: Not all player info columns found. Missing: {list(set(player_info_cols) - set(existing_player_info_cols))}. Player info might be incomplete.")
        player_info = final_df[existing_player_info_cols].copy()


        print(f"\nDataFrame for clustering created with shape: {df_clustering.shape}")
        print("Columns for clustering:")
        print(df_clustering.columns.tolist())


        # 5. Feature Scaling
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(df_clustering)
        df_scaled = pd.DataFrame(scaled_features, columns=df_clustering.columns)

        print("\n--- Scaled Features Head ---")
        print(df_scaled.head())


        # 6. Save Processed Data for Clustering (scaled and player info)
        df_scaled.to_csv('../data/processed/nba_2025_player_stats_scaled_for_clustering.csv', index=False)
        player_info.to_csv('../data/processed/nba_2025_player_info.csv', index=False)
        print("\nCleaned and scaled data for clustering saved.")
        print("Player info saved.")

    else:
        print("✗ Cannot perform second merge - missing required columns")
else:
    print("✗ Cannot perform first merge - missing required columns")

DataFrames loaded successfully.

✓ First merge completed successfully
✓ Second merge completed successfully

--- Merged DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737 entries, 0 to 736
Data columns (total 90 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Rk_per100                  736 non-null    object 
 1   Player                     736 non-null    object 
 2   Age                        735 non-null    float64
 3   Team                       735 non-null    object 
 4   Pos_per100                 735 non-null    object 
 5   G_per100                   735 non-null    float64
 6   GS_per100                  735 non-null    float64
 7   MP_per100                  735 non-null    float64
 8   FG                         735 non-null    float64
 9   FGA                        735 non-null    float64
 10  FG%                        732 non-null    float64
 11  3P                     

/var/folders/_8/gvzx8lwx3xd08y47f2s1y2rc0000gn/T/ipykernel_35224/1721265537.py:136: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(0, inplace=True)
/var/folders/_8/gvzx8lwx3xd08y47f2s1y2rc0000gn/T/ipykernel_35224/1721265537.py:141: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beha

In [7]:
# --- Initial Data Cleaning and Feature Selection ---

# First, let's make sure we have the merged DataFrame
# If final_df is not defined, we need to run the previous cell first
try:
    final_df
except NameError:
    print("Error: final_df is not defined. Please run the previous cell first to load and merge the data.")
    print("Make sure to run the cells in order: 1) Load and merge data, 2) Clean and prepare data")
    exit()

# 1. Handle redundant/duplicate columns from merging
# Identify columns that are duplicates (e.g., 'MP_per100', 'MP_adv', 'G_per100', 'G_adv', etc.)
# It's good practice to inspect `final_df.columns` to see what you have.

# Example: Drop redundant minute played columns, keeping one (e.g., from per_100_poss)
columns_to_drop_after_merge = []
for col in final_df.columns:
    if col.endswith('_adv') or col.endswith('_shooting') and col not in ['Player', 'Team', 'Age']: # Updated column names
        original_col_name = col.replace('_adv', '').replace('_shooting', '')
        if original_col_name in final_df.columns and original_col_name != col: # Check if original (un-suffixed) exists
            # We assume the first column (from df_per_100) is the one to keep, drop the others
            columns_to_drop_after_merge.append(col)
        elif original_col_name + '_per100' in final_df.columns and original_col_name != col:
             # If a column like 'MP' from advanced stats is now 'MP_adv' and 'MP_per100' exists, drop 'MP_adv'
            columns_to_drop_after_merge.append(col)


# Common columns that often exist in multiple tables but you only need one version:
common_stats = ['G', 'GS', 'MP'] # Games, Games Started, Minutes Played

for stat in common_stats:
    if f"{stat}_adv" in final_df.columns and f"{stat}_per100" in final_df.columns:
        columns_to_drop_after_merge.append(f"{stat}_adv") # Keep the per100 version
    elif f"{stat}_shooting" in final_df.columns and f"{stat}_per100" in final_df.columns:
         columns_to_drop_after_merge.append(f"{stat}_shooting") # Keep the per100 version


final_df.drop(columns=columns_to_drop_after_merge, inplace=True, errors='ignore') # Use errors='ignore' in case some aren't present


# 2. Filter out players with insufficient playing time
# This is crucial for meaningful archetypes. For a full season, 500-700 minutes is a good lower bound.
# Rookies might have less, so consider your minimum. A common threshold is 15-20 games OR 300-500 minutes.
min_mp_threshold = 500 # Example: Minimum 500 minutes played
final_df = final_df[final_df['MP'] >= min_mp_threshold].copy() # .copy() to avoid SettingWithCopyWarning

print(f"\nFiltered to players with >= {min_mp_threshold} minutes. New shape: {final_df.shape}")

# 3. Select features for clustering
# This is where your basketball knowledge comes in!
# Aim for a diverse set of stats that capture different aspects of play.
# Avoid highly correlated features initially to prevent redundancy (though PCA can handle this later).

# Example feature selection (you'll refine this extensively!)
features_for_clustering = [
    # Per 100 Possessions (rate stats are generally best for clustering)
    'PTS', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'PF', # Basic volume stats
    'FG%', '3P%', '2P%', 'FT%', # Shooting efficiency
    # Advanced Stats
    'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
    'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'BPM', 'VORP',
    # Shooting Stats (some might be redundant with advanced stats, choose carefully)
    'Dist.', # Average Shot Distance
    '2P_FG%', '0-3_FG%', '3-10_FG%', '10-16_FG%', '16-3P_FG%', # FG% by distance
    'Att_3P', 'Att_2P', # Shot attempts (might already be covered by USG% but can be useful)
]

# Ensure all selected features exist in your final DataFrame
# It's common to find some columns might not exist if they were merged with suffixes
# or were only present in one of the original CSVs.
# Remove any features from your list that aren't in final_df.columns
actual_features = [col for col in features_for_clustering if col in final_df.columns]
missing_features = [col for col in features_for_clustering if col not in final_df.columns]

if missing_features:
    print(f"\nWarning: The following desired features are missing and will be excluded: {missing_features}")

df_clustering = final_df[actual_features].copy()

# Add Player and Team for later interpretation (but exclude from clustering features)
player_info = final_df[['Player', 'Team', 'Pos', 'Age']].copy() # Updated column names

print(f"\nDataFrame for clustering created with shape: {df_clustering.shape}")
print("Columns for clustering:")
print(df_clustering.columns.tolist())


Filtered to players with >= 500 minutes. New shape: (375, 82)


DataFrame for clustering created with shape: (375, 30)
Columns for clustering:
['PTS', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'PF', 'FG%', '3P%', '2P%', 'FT%', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'BPM', 'VORP', 'Dist.']


In [8]:
# --- Handle Missing Values ---
print("\n--- Missing Value Check (before imputation) ---")
print(df_clustering.isnull().sum())

# Strategy: Impute missing numerical values.
# For percentages/ratios: often 0 makes sense if it's truly a "no attempts" scenario.
# For other stats: mean or median imputation can be used.
# A robust approach is to check each column individually.

for col in df_clustering.columns:
    if df_clustering[col].isnull().any():
        # A common imputation for stats where NaN means 'no attempts' is 0
        if 'FG%' in col or '3P%' in col or 'FT%' in col or 'Ar' in col or 'Dist' in col: # Check for percentage/attempt ratio columns
            df_clustering[col].fillna(0, inplace=True)
        else:
            # For other numerical stats, median is often more robust to outliers than mean
            median_val = df_clustering[col].median()
            df_clustering[col].fillna(median_val, inplace=True)
            print(f"Imputed missing values in '{col}' with median: {median_val}")

print("\n--- Missing Value Check (after imputation) ---")
print(df_clustering.isnull().sum())


--- Missing Value Check (before imputation) ---
PTS      0
AST      0
TRB      0
STL      0
BLK      0
TOV      0
PF       0
FG%      0
3P%      0
2P%      0
FT%      0
PER      0
TS%      0
3PAr     0
FTr      0
ORB%     0
DRB%     0
TRB%     0
AST%     0
STL%     0
BLK%     0
TOV%     0
USG%     0
OWS      0
DWS      0
WS       0
WS/48    0
BPM      0
VORP     0
Dist.    0
dtype: int64

--- Missing Value Check (after imputation) ---
PTS      0
AST      0
TRB      0
STL      0
BLK      0
TOV      0
PF       0
FG%      0
3P%      0
2P%      0
FT%      0
PER      0
TS%      0
3PAr     0
FTr      0
ORB%     0
DRB%     0
TRB%     0
AST%     0
STL%     0
BLK%     0
TOV%     0
USG%     0
OWS      0
DWS      0
WS       0
WS/48    0
BPM      0
VORP     0
Dist.    0
dtype: int64


In [9]:
from sklearn.preprocessing import StandardScaler

# --- Feature Scaling ---
# StandardScaler (Z-score normalization) is generally preferred for K-Means.
# It transforms data to have a mean of 0 and a standard deviation of 1.
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_clustering)

# Convert back to a DataFrame for easier handling, keeping column names
df_scaled = pd.DataFrame(scaled_features, columns=df_clustering.columns)

print("\n--- Scaled Features Head ---")
print(df_scaled.head())


--- Scaled Features Head ---
        PTS       AST       TRB       STL       BLK       TOV        PF  \
0  0.228359 -0.064184 -1.194319 -0.814994 -0.386370 -0.422695 -1.575076   
1 -0.587625  0.995455  0.901444  0.571837 -0.633199  0.034899 -0.419198   
2  2.239178  0.389947 -0.321084 -0.121579 -0.139541  1.499199 -1.130508   
3  1.743759  1.600963 -0.894924 -0.814994 -0.880028  1.133124 -0.330285   
4  1.423194  2.622758 -0.221286  0.745191 -0.016126  3.055018 -0.863767   

        FG%       3P%       2P%  ...      BLK%      TOV%      USG%       OWS  \
0  0.438961  0.269348  0.795891  ... -0.421257 -0.676091  0.072863  1.032157   
1  0.802951  0.064469  1.158172  ... -0.678960  0.871150 -0.686125  1.923393   
2 -0.332696  0.669351 -0.607948  ... -0.163555 -0.062530  2.155666  1.503988   
3 -0.128862  0.054713  0.010949  ... -0.936662 -0.089206  1.784998  2.290373   
4 -0.871400  0.249836 -1.045704  ...  0.029722  1.538064  1.837950  1.189434   

        DWS        WS     WS/48       

In [10]:
# Save the cleaned and scaled data for clustering
df_scaled.to_csv('../data/raw/nba_2025_player_stats_scaled_for_clustering.csv', index=False)
player_info.to_csv('../data/raw/nba_2025_player_info.csv', index=False) # Keep player info separate
print("\nCleaned and scaled data saved as 'nba_2025_player_stats_scaled_for_clustering.csv'")
print("Player info saved as 'nba_2025_player_info.csv'")


Cleaned and scaled data saved as 'nba_2025_player_stats_scaled_for_clustering.csv'
Player info saved as 'nba_2025_player_info.csv'
